In [14]:
import os

from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

In [15]:
CLIENT_SECRETS_FILE = "client_secret.json" 

In [16]:
SCOPES = ['https://www.googleapis.com/auth/youtube.force-ssl']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

def get_authenticated_service():
  flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
  credentials = flow.run_console()
  return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)

os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'
service = get_authenticated_service()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=288103088130-0ksj54vqcd2eoj9sfka0l7v112405fur.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=jVQjb4cq1U3MfjzFOnFp752JGrn6ai&prompt=consent&access_type=offline&code_challenge=ejgGG7RkyaW7RR0JZv96jLQoR_hWpXVtOXuhZlLGg1Y&code_challenge_method=S256
Enter the authorization code: 4/qQHqXZdVN0ND2X-WBwR8-9hnrY-IaPL5etA6auIzWeJGcI9abfdIqdc


In [17]:
query = 'hema'

query_results = service.search().list(
        part = 'snippet',
        q = query,
        order = 'relevance', # You can consider using viewCount
        maxResults = 20,
        type = 'video', # Channels might appear in search results
        relevanceLanguage = 'en',
        safeSearch = 'moderate',
        ).execute()

In [18]:
query_results


{'kind': 'youtube#searchListResponse',
 'etag': '"8jEFfXBrqiSrcF6Ee7MQuz8XuAM/Fn9N2BrUV4tEfLvm-mFowuGj48Y"',
 'nextPageToken': 'CBQQAA',
 'regionCode': 'CO',
 'pageInfo': {'totalResults': 1000000, 'resultsPerPage': 20},
 'items': [{'kind': 'youtube#searchResult',
   'etag': '"8jEFfXBrqiSrcF6Ee7MQuz8XuAM/mTl5yKiIZ-rUPFWZGv58T8frwJc"',
   'id': {'kind': 'youtube#video', 'videoId': '91IIARM5lVs'},
   'snippet': {'publishedAt': '2018-11-12T16:24:58.000Z',
    'channelId': 'UCKUAug0soGCJ9y8bQ0Vx0qw',
    'title': 'HEMA | Swordfish 2018 - Open Longsword FINALS',
    'description': 'Sergey Kultaev vs Antoni Olbrychski [Men\'s Longsword Finals] Round 1: 5:20 Round 2: 9:59 "Sword of Destiny" music by Jillian Aversa: ...',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/91IIARM5lVs/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/91IIARM5lVs/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i

In [19]:
video_id = []
channel = []
video_title = []
video_desc = []
for item in query_results['items']:
    video_id.append(item['id']['videoId'])
    channel.append(item['snippet']['channelTitle'])
    video_title.append(item['snippet']['title'])
    video_desc.append(item['snippet']['description'])

In [20]:
video_id_pop = []
channel_pop = []
video_title_pop = []
video_desc_pop = []
comments_pop = []
comment_id_pop = []
reply_count_pop = []
like_count_pop = []

from tqdm import tqdm
for i, video in enumerate(tqdm(video_id, ncols = 100)):
    response = service.commentThreads().list(
                    part = 'snippet',
                    videoId = video,
                    maxResults = 100, # Only take top 100 comments...
                    order = 'relevance', #... ranked on relevance
                    textFormat = 'plainText',
                    ).execute()
    
    comments_temp = []
    comment_id_temp = []
    reply_count_temp = []
    like_count_temp = []
    for item in response['items']:
        comments_temp.append(item['snippet']['topLevelComment']['snippet']['textDisplay'])
        comment_id_temp.append(item['snippet']['topLevelComment']['id'])
        reply_count_temp.append(item['snippet']['totalReplyCount'])
        like_count_temp.append(item['snippet']['topLevelComment']['snippet']['likeCount'])
    comments_pop.extend(comments_temp)
    comment_id_pop.extend(comment_id_temp)
    reply_count_pop.extend(reply_count_temp)
    like_count_pop.extend(like_count_temp)
    
    video_id_pop.extend([video_id[i]]*len(comments_temp))
    channel_pop.extend([channel[i]]*len(comments_temp))
    video_title_pop.extend([video_title[i]]*len(comments_temp))
    video_desc_pop.extend([video_desc[i]]*len(comments_temp))
    
query_pop = [query] * len(video_id_pop)

100%|███████████████████████████████████████████████████████████████| 20/20 [00:23<00:00,  1.14it/s]


In [22]:
import pandas as pd

output_dict = {
        'Query': query_pop,
        'Channel': channel_pop,
        'Video Title': video_title_pop,
        'Video Description': video_desc_pop,
        'Video ID': video_id_pop,
        'Comment': comments_pop,
        'Comment ID': comment_id_pop,
        'Replies': reply_count_pop,
        'Likes': like_count_pop,
        }

output_df = pd.DataFrame(output_dict, columns = output_dict.keys())

In [23]:
output_df

,Query,Channel,Video Title,Video Description,Video ID,Comment,Comment ID,Replies,Likes
0,hema,ViciouSExorcisT,HEMA | Swordfish 2018 - Open Longsword FINALS,Sergey Kultaev vs Antoni Olbrychski [Men's Lon...,91IIARM5lVs,"I just love the video tags on this: katana, ni...",Ugw6BmsCPoe2G37woop4AaABAg,5,187
1,hema,ViciouSExorcisT,HEMA | Swordfish 2018 - Open Longsword FINALS,Sergey Kultaev vs Antoni Olbrychski [Men's Lon...,91IIARM5lVs,>Hears Witcher 3\n>AWWWWWWWWWW SHIET\n\nI comp...,UgyHfELJMwxO_l_dyGR4AaABAg,0,100
2,hema,ViciouSExorcisT,HEMA | Swordfish 2018 - Open Longsword FINALS,Sergey Kultaev vs Antoni Olbrychski [Men's Lon...,91IIARM5lVs,"Steel for humans, silver for monsters.",UgxbM7UiMcjM6QESw8B4AaABAg,0,60
3,hema,ViciouSExorcisT,HEMA | Swordfish 2018 - Open Longsword FINALS,Sergey Kultaev vs Antoni Olbrychski [Men's Lon...,91IIARM5lVs,The form of these competitors is absolutely be...,Ugxm4HSVjXHWmPBkY6h4AaABAg,1,34
4,hema,ViciouSExorcisT,HEMA | Swordfish 2018 - Open Longsword FINALS,Sergey Kultaev vs Antoni Olbrychski [Men's Lon...,91IIARM5lVs,"Not a fan of the scoring system, mostly a grea...",UgzXp2aXT8ckj9hnFFJ4AaABAg,9,45
5,hema,ViciouSExorcisT,HEMA | Swordfish 2018 - Open Longsword FINALS,Sergey Kultaev vs Antoni Olbrychski [Men's Lon...,91IIARM5lVs,Wait... Do I hear Witcher 3 music?!?!,Ugw2BTtkQ4zdAPjXG9B4AaABAg,5,36
6,hema,ViciouSExorcisT,HEMA | Swordfish 2018 - Open Longsword FINALS,Sergey Kultaev vs Antoni Olbrychski [Men's Lon...,91IIARM5lVs,"Wow, collecting footage, photos, and informati...",UgzOR4AUhhP-WwHFPfd4AaABAg,1,32
7,hema,ViciouSExorcisT,HEMA | Swordfish 2018 - Open Longsword FINALS,Sergey Kultaev vs Antoni Olbrychski [Men's Lon...,91IIARM5lVs,There sure are a lot of Russians in this tourn...,Ugw4GkzvbA0rps-_3LV4AaABAg,1,29
8,hema,ViciouSExorcisT,HEMA | Swordfish 2018 - Open Longsword FINALS,Sergey Kultaev vs Antoni Olbrychski [Men's Lon...,91IIARM5lVs,This sport could really catch on... but it wou...,Ugw0ac_woOm2PkqYLYd4AaABAg,0,14
9,hema,ViciouSExorcisT,HEMA | Swordfish 2018 - Open Longsword FINALS,Sergey Kultaev vs Antoni Olbrychski [Men's Lon...,91IIARM5lVs,That was amazing! Great fight! Both showed rea...,UgyeB4l0dPZepxgMBy94AaABAg,3,40
